## Audio Processing using Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa

In [ ]:
audio_path = "audio.mp3"

In [ ]:
y, sr = librosa.load(audio_path, sr=None)

In [ ]:
y

In [ ]:
y[:100]

In [ ]:
y.shape

In [ ]:
sr

In [ ]:
plt.figure(figsize=(14, 5), dpi=150)
plt.plot(y)
plt.xlabel("Time - Samples")
plt.ylabel("Amplitude");

In [ ]:
len(y) / sr

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(data=y, rate=sr)

In [ ]:
# DFT - Discrete Fourier Transform
window = np.hanning(len(y))
windowed_input = y * window
dft = np.fft.rfft(windowed_input)

In [ ]:
dft

In [ ]:
plt.plot(dft)
plt.title("Discrete Fourier Transform");

In [ ]:
amplitude = np.abs(dft)

In [ ]:
plt.plot(amplitude);

In [ ]:
amplitude_db = librosa.amplitude_to_db(amplitude, ref=np.max)

In [ ]:
frequency = librosa.fft_frequencies(sr=sr, n_fft=len(y))

In [ ]:
plt.figure(figsize=(15, 4), dpi=150)
plt.plot(frequency, amplitude_db)
plt.xlabel("Freq Hz")
plt.ylabel("Amp dB");
plt.xscale("log"); # logarithm

In [ ]:
D = librosa.stft(y)
D

In [ ]:
D_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

In [ ]:
plt.figure(figsize=(14, 5), dpi=150)
librosa.display.specshow(D_db, sr=sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")

In [ ]:
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)

plt.figure(figsize=(14, 5), dpi=150)
librosa.display.specshow(S_dB, sr=sr, x_axis="time", y_axis="log", fmax=8000)
plt.colorbar(format="%+2.0f dB")

## Import required libraries - Audio Classification

In [ ]:
import torch, transformers, torchaudio
print(torch.__version__)
print(transformers.__version__)
print(torchaudio.__version__)

## Audio Classification

In [ ]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification

In [ ]:
import librosa

In [ ]:
audio_path = "audio.mp3"

In [ ]:
y, sr = librosa.load(audio_path, sr=None)

In [ ]:
y

In [ ]:
sr

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(data=audio_path)

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

In [ ]:
result = feature_extractor(y, return_tensors="pt")

In [ ]:
result["input_values"]

In [ ]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

In [ ]:
model

In [ ]:
result["input_values"]

In [ ]:
prediction_logits = model(result["input_values"]).logits

In [ ]:
prediction_logits

In [ ]:
predicted_class_ids = torch.argmax(prediction_logits, dim=-1).item()

In [ ]:
predicted_class_ids

In [ ]:
model.config.id2label

In [ ]:
model.config.id2label[predicted_class_ids]

## Converting Audio to Text

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("automatic-speech-recognition")

In [ ]:
pipe

In [ ]:
pipe("audio.mp3")

## Converting Text to Audio

In [ ]:
from transformers import pipeline
pipe = pipeline("text-to-speech")

In [ ]:
text = "I like coding in python"

In [ ]:
text

In [ ]:
output = pipe(text)

In [ ]:
output

In [ ]:
output["audio"]

In [ ]:
output["audio"].shape

In [ ]:
plt.plot(output["audio"].squeeze());

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(data=output["audio"], rate=output["sampling_rate"])

In [ ]:
from pydub import AudioSegment

In [ ]:
audio_seg = AudioSegment(output["audio"].tobytes(),
                         frame_rate=output["sampling_rate"],
                         sample_width=output["audio"].dtype.itemsize,
                         channels=1)

In [ ]:
audio_seg.export("my_audio_saved.mp3", format="mp3")